In [ ]:
import os
import sys

import pandas as pd
import numpy as np

In [ ]:
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('seaborn-bright')
plt.rcParams['figure.figsize'] = [7, 5]
plt.rcParams['font.size'] = 12

In [ ]:
def resample_data(df, freq):
    
    df_sums = df.copy(deep=True)
    df_means = df.copy(deep=True)
    
    df_sums = df_sums.resample(freq).sum()
    df_means = df_means.resample(freq).mean()
    
    for col in df_sums.columns:
        if '[kWh]' in col:
            # drop col from means
            df_means.drop(col, axis=1, inplace=True)
        else:
            # drop col from sums
            df_sums.drop(col, axis=1, inplace=True)
            
    df_temp = pd.concat([df_sums, df_means], axis=1, sort=True)
    return df_temp.reindex(sorted(df_temp.columns), axis=1)

In [ ]:
def process_results(path):
    
    df = pd.read_csv(path)
    
    df['Total Imm. Elec. [W]'] = df['SwedishHP:SVENSKA VARMMEPUMPE:Electrical Usage for Immersion Heater for Heating [W]'] + \
                                 df['SwedishHP:SVENSKA VARMMEPUMPE:Electrical Usage for Immersion Heater fow Water Heating [W]']
    
    df['Total HP Elec. [W]'] = df['SwedishHP:SVENSKA VARMMEPUMPE:Electrical Usage for Heating [W]'] + \
                               df['SwedishHP:SVENSKA VARMMEPUMPE:Electrical Usage for Water Heating [W]']
    
    df['COP [-]'] = (df['SwedishHP:SVENSKA VARMMEPUMPE:Heating Load [W]'] + \
                     df['SwedishHP:SVENSKA VARMMEPUMPE:Water Heating Load [W]']) / \
                    (df['Total HP Elec. [W]'] + df['Total Imm. Elec. [W]'])
    
    for col in df.columns:
        if '[W]' in col:
            # convert W to kWh. assuming a 1 hour timestep
            df[col] = df[col]  / 1000
            
            # rename
            new_name = col.replace('[W]', '[kWh]')
            df.rename(columns={col: new_name}, inplace=True)
            
    # get rid of leap days --- FML
    periods = pd.period_range('2019-01-01 00:00:00', '2038-12-31 23:00:00', freq='H')
    is_leap_day = (periods.month == 2) & (periods.day == 29)
    periods_without_leap = periods[~is_leap_day]
    timestamps = periods_without_leap.to_timestamp()
    
    df.drop('Date/Time', axis=1, inplace=True)
    df.drop(df.head(1).index, inplace=True)
    df['Date/Time'] = timestamps
    df.set_index('Date/Time', inplace=True)
        
    df_mo = resample_data(df, 'M')
    df_ann = resample_data(df_mo, 'A')
    
    return df, df_mo, df_ann

In [ ]:
df_s, df_s_mo, df_s_ann = process_results('out_20-yr_self.csv')
df_c1, df_c1_mo, df_c1_ann = process_results('out_20-yr_1-cross_start-5-yr.csv')
df_c2, df_c2_mo, df_c2_ann = process_results('out_20-yr_2-cross_start-5-yr.csv')
df_c3, df_c3_mo, df_c3_ann = process_results('out_20-yr_3-cross_start-5-yr.csv')

In [ ]:
df_s_mo.columns

In [ ]:
def plot_ann_data(col, xlabel, y1_label, y2_label, percent_change=False, save_name=False):

    fig = plt.figure(dpi=200)

    ax = fig.add_subplot(211)

    x = np.arange(1, 21)

    ax.plot(x, df_s_ann[col], label='None', c='k')
    ax.plot(x, df_c1_ann[col], label='1', c='r')
    ax.plot(x, df_c2_ann[col], label='2', c='g')
    ax.plot(x, df_c3_ann[col], label='3', c='b')
    plt.legend()
    plt.grid()
    plt.ylabel(y1_label)

    ax2 = fig.add_subplot(212)
    
    if percent_change:
        err1 = (df_c1_ann[col] - df_s_ann[col]) / df_s_ann[col] * 100
        err2 = (df_c2_ann[col] - df_s_ann[col]) / df_s_ann[col] * 100
        err3 = (df_c3_ann[col] - df_s_ann[col]) / df_s_ann[col] * 100
    else:
        err1 = df_c1_ann[col] - df_s_ann[col]
        err2 = df_c2_ann[col] - df_s_ann[col]
        err3 = df_c3_ann[col] - df_s_ann[col]

    ax2.plot(x, err1, c='r')
    ax2.plot(x, err2, c='g')
    ax2.plot(x, err3, c='b')
    plt.ylabel(y2_label)
    plt.xlabel(xlabel)
    plt.grid()
    
    if save_name:
        plt.savefig(save_name, bbox_inches='tight')

    plt.show()

In [ ]:
plot_ann_data('SwedishHP:SVENSKA VARMMEPUMPE:Inlet Temp. [C]', 'Years', 'Ave. HP Inlet Temp. [C]', r'$\Delta$ T',  save_name='HP-Inlet-Temp_Start-5-yr.PNG')

In [ ]:
plot_ann_data('COP [-]', 'Years', 'Ave. COP [-]', '% Change', percent_change=True,  save_name='COP_Start-5-yr.PNG')

In [ ]:
plot_ann_data('Total HP Elec. [kWh]', 'Years', 'Tot. HP Elec. [kWh]', '% Change', percent_change=True,  save_name='HP-Elec_Start-5-yr.PNG')

In [ ]:
plot_ann_data('Total Imm. Elec. [kWh]', 'Years', 'Tot. HTR Elec. [kWh]', '% Change', percent_change=True,  save_name='HTR_Elec_Start-5-yr.PNG')

In [ ]:
def plot_ann_data_src_side(col, xlabel, y1_label, y2_label, save_name=False):

    fig = plt.figure(dpi=200)

    ax = fig.add_subplot(211)

    x = np.arange(1, 21)

    ax.plot(x, abs(df_s_ann[col]), label='None', c='k')
    ax.plot(x, abs(df_c1_ann[col]), label='1', c='r')
    ax.plot(x, abs(df_c2_ann[col]), label='2', c='g')
    ax.plot(x, abs(df_c3_ann[col]), label='3', c='b')
    plt.legend()
    plt.grid()
    plt.ylabel(y1_label)

    ax2 = fig.add_subplot(212)
    
    err1 = (df_c1_ann[col] - df_s_ann[col]) / df_s_ann[col] * 100
    err2 = (df_c2_ann[col] - df_s_ann[col]) / df_s_ann[col] * 100
    err3 = (df_c3_ann[col] - df_s_ann[col]) / df_s_ann[col] * 100

    ax2.plot(x, err1, c='r')
    ax2.plot(x, err2, c='g')
    ax2.plot(x, err3, c='b')
    plt.ylabel(y2_label)
    plt.xlabel(xlabel)
    plt.grid()
    
    if save_name:
        plt.savefig(save_name, bbox_inches='tight')

    plt.show()

In [ ]:
plot_ann_data_src_side('SwedishHP:SVENSKA VARMMEPUMPE:Source-side Heat Rate [kWh]', 'Years', 'Tot. HP. Src-Side [kWh]', '% Change',  save_name='HP-SRC-Side_Start-5-yr.PNG')

In [ ]:
plot_ann_data_src_side('SwedishHP:SVENSKA VARMMEPUMPE:Load-side Heat Rate [kWh]', 'Years', 'Tot. HP. Load-Side [kWh]', '% Change', save_name='HP-Load-Side_Start-5-yr.PNG')

In [ ]:
plot_ann_data('GroundHeatExchangerLTS:SELF-GHE:BH Resist. [m-K/W]', 'Years', r'$R_b$ [m-K/W]', '% Change', percent_change=True, save_name='Rb_Start-5-yr.PNG')

In [ ]:
plot_ann_data('GroundHeatExchangerLTS:SELF-GHE:Borehole Wall Temp. [C]', 'Years', 'BH Wall Temp. [C]', r'$\Delta$ T', save_name='BH_Wall_Temp_Start-5-yr.PNG')

In [ ]:
df_s_mo_ann_mean = df_s_mo.resample('A').mean()

In [ ]:
df_c3_mo_mean = df_c3_mo.resample('A').mean()

In [ ]:
def plot_montly_mean_data(col, xlabel, y1_label, save_name=False):

    fig = plt.figure(dpi=200)

    ax = fig.add_subplot(111)
    
    xday = np.linspace(0, 20, num=175200)
    xmo = np.arange(1/12, 20, 1/12)
    xann = np.arange(1, 21)
    ax.plot(xday, df_s[col], label='Hourly')
    ax.plot(xmo, df_s_mo[col], label='Monthly')
    ax.plot(xann, df_s_mo_ann_mean[col], label='Annual')
    plt.legend()
    plt.grid()
    plt.ylabel(y1_label)
    
    plt.xlabel('Years')
    
    if save_name:
        plt.savefig(save_name, bbox_inches='tight')

    plt.show()

In [ ]:
plot_montly_mean_data('SwedishHP:SVENSKA VARMMEPUMPE:Inlet Temp. [C]', 'Years', 'HP Inlet Temp. [C]', save_name='Self_Houly_Monthly_Annual_Inlet_Temp.PNG')

In [ ]:
fig = plt.figure(dpi=200)

ax = fig.add_subplot(111)

x = np.arange(0, 48, 1)

start = '2019-02-01 00:00:00'
end =   '2019-02-02 23:00:00'

ln1 = ax.plot(x, df_s['SwedishHP:SVENSKA VARMMEPUMPE:Inlet Temp. [C]'].loc[start:end], label=r'$T_{HP,in,src}$ [C]', c='g')
ln2 = ax.plot(x, df_s['SwedishHP:SVENSKA VARMMEPUMPE:Outlet Temp. [C]'].loc[start:end], label=r'$T_{HP,out,src}$ [C]', c='b')
ln3 = ax.plot(x, df_s['SwedishHP:SVENSKA VARMMEPUMPE:Outdoor Air Temp. [C]'].loc[start:end], label=r'$T_{oda}$ [C]', c='k')

ax.legend(loc=2)
ax.set_ylabel('Temperature [C]')
ax.set_xlabel('Hour')

ax2 = ax.twinx()

ln4 = ax2.plot(x, df_s['SwedishHP:SVENSKA VARMMEPUMPE:Heating Load [kWh]'].loc[start:end], label=r'$Q_{htg}$ [kW]', c='r', linestyle='--')
ln5 = ax2.plot(x, df_s['SwedishHP:SVENSKA VARMMEPUMPE:Water Heating Load [kWh]'].loc[start:end], label=r'$Q_{wtr}$ [kW]', c='c', linestyle='-.')

# lns = ln1+ln2+ln3+ln4+ln5
# labs = [l.get_label() for l in lns]

ax2.legend(loc=1)
ax2.set_ylabel('Load [kW]')

plt.xticks(np.arange(0, 48+6, 6))

plt.grid()

plt.savefig('Feb_Temps_Loads_Self.PNG')

plt.show()

In [ ]:
np.arange(0, 48+6, 6)